## Example 15 - Emission footprint of IWT (try out versie)

In [33]:
import opentnsim
print('This notebook has been tested with OpenTNSim version {} - branch: Afstuderen_LoesSegers'.format(opentnsim.__version__))
print('This is a try-out file')

This notebook has been tested with OpenTNSim version 1.0.0 - branch: Afstuderen_LoesSegers
This is a try-out file


In [34]:
# package(s) related to time, space and id
import datetime, time
import platform
import random
import os

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import pyproj
import shapely.geometry
from simplekml import Kml, Style
import folium

# package(s) for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from osgeo import ogr, osr

# OpenTNSIM
import opentnsim.core as core
import opentnsim.graph_module as graph_module
import opentnsim.plot as plot
import opentnsim.model as model

# Used for mathematical functions
import math             
import json

# Used for making the graph to visualize our problem
import networkx as nx  

# Graph location
location_graph = "Shape-Files/Vaarwegvakken"
name_graph = "Vaarwegvakken.shp"

# Vessel database
location_vessel_database = "Vessels\Vessel-database-2.csv"

### Initiate Simpy environment

In [35]:
# Start simpy environment
simulation_start = datetime.datetime.now()
env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))

#A simulation environment manages the simulation time as well as the scheduling and processing of events. 
#It also provides means to step through or execute the simulation

#SimpPy --> discrete event simulation

### Load and transform graph

**Important**: 

If you use windows and get the following error "ImportError: read_shp requires OGR: http://www.gdal.org/", you probably have [this issue](https://github.com/conda-forge/gdal-feedstock/issues/219). Solving it is possible by running the following commands in your terminal (as explained [here](https://gis.stackexchange.com/questions/294231/installing-gdal-with-anaconda)):

```bash
#Create a new virtual environment
conda create -n testgdal -c conda-forge gdal vs2015_runtime=14

#Activate virtual environment
activate testgdal

#Open Jupyter notebook
jupyter notebook
```

In [36]:
FG = nx.read_shp(os.path.join(location_graph, name_graph), simplify=True)

# The read_shp creates a directed graph with single edges
# We require a directed graph but two-way traffic

FG = FG.to_undirected()
FG = FG.to_directed()

#FG = de graaf

In [37]:
def transform_projection(location_graph, name_graph):
    driver = ogr.GetDriverByName("ESRI Shapefile")
    dataset = driver.Open(os.path.join(location_graph, name_graph))

    # from Layer
    inSpatialRef = dataset.GetLayer().GetSpatialRef()

    # Set up the coordinate reference we want to use, WGS84 - World Geodetic System 1984
    outSpatialRef = osr.SpatialReference()
    outSpatialRef.ImportFromEPSG(4326)

    # Transform the coordinates
    transform = osr.CoordinateTransformation(inSpatialRef, outSpatialRef)
    
    return transform

In [38]:
def change_projection(transform, point):
    point = ogr.CreateGeometryFromWkt(str(point))
    
    point.Transform(transform)
    point.ExportToWkt()
    
    return point.GetX(), point.GetY()

In [39]:
#FG.nodes

In [40]:
transform = transform_projection(location_graph, name_graph)

FG_new = nx.DiGraph()
nodes_dict = {}

for i, node in enumerate(FG.nodes(data = True)):
    coordinates = change_projection(transform, shapely.geometry.Point(list(FG.nodes)[i][0], list(FG.nodes)[i][1]))
    name = "({:f}, {:f})".format(coordinates[0], coordinates[1])
    geometry = shapely.geometry.Point(coordinates[0], coordinates[1])
    
    nodes_dict[list(FG.nodes)[i]] = name
    FG_new.add_node(name, name = name, Position = coordinates, geometry = geometry, Old = node[1])
    
for edge in FG.edges(data = True):
    node_1 = nodes_dict[edge[0]]
    node_2 = nodes_dict[edge[1]]
    
    VRT_NAAM = edge[2]["VRT_NAAM"]
    VWG_NAAM = edge[2]["VWG_NAAM"]
    BEGKM =  edge[2]["BEGKM"]
    ENDKM =  edge[2]["ENDKM"]
    DIST = np.abs(float(BEGKM) - float(ENDKM))
    
    LINE = (json.loads(edge[2]["Json"])["coordinates"])
    LineString = []
    for coordinates in LINE:
        LineString.append(change_projection(transform, shapely.geometry.Point(coordinates[0], coordinates[1])))
    
    FG_new.add_edge(node_1, node_2, LineString = shapely.geometry.LineString(LineString), 
                    VRT_NAAM = VRT_NAAM, VWG_NAAM = VWG_NAAM, BEGKM = BEGKM, ENDKM = ENDKM, DIST = DIST)

if nx.info(FG) == nx.info(FG_new):
    print("Succes!")

Succes!


In [41]:
#FG_new.edges(data=True)

### Select only relevant area

In [42]:
# North-East
NE = (4.54, 51.75)

# South-East
SE = (4.54, 51.60)

# South-West
SW = (4.20, 51.60)

# North-West
NW = (4.20, 51.75)

#Polygoon of AIS data RWS
#NE = (4.694, 51.929)
#SE = (4.694, 51.327)
#SW = (3.793, 51.327)
#NW = (3.793, 51.929)


polygon = shapely.geometry.Polygon([NE, SE, SW, NW])

In [43]:
nodes = []
edges = []

for edge in FG_new.edges(data = True):
    node_1 = FG_new.nodes[edge[0]]
    node_2 = FG_new.nodes[edge[1]]
    
    if node_1["geometry"].within(polygon) or node_2["geometry"].within(polygon):
        nodes.append(node_1)
        nodes.append(node_2)
        edges.append(edge)

In [44]:
FG_new = nx.DiGraph ()

for node in nodes:
    FG_new.add_node(node["name"], name = node["name"], Position = node["Position"], geometry = node["geometry"])

for edge in edges:
    FG_new.add_edge(edge[0], edge[1], Info = edge[2])

### Show on map

In [45]:
# Browser
m = folium.Map(location=[51.7, 4.4], zoom_start = 12)

for edge in FG_new.edges(data = True):
#     print(edge[2]["Info"]["VWG_NAAM"])
    points_x = list(edge[2]["Info"]["LineString"].coords.xy[0])
    points_y = list(edge[2]["Info"]["LineString"].coords.xy[1])
    
    line = []
    for i, _ in enumerate(points_x):
        line.append((points_y[i], points_x[i]))
    
    if edge[2]["Info"]["VWG_NAAM"] in ["Voorhavens Jachtensluis", "Voorhavens Volkeraksluizen"]:
        folium.PolyLine(line, color = "red", weight = 5, popup = edge[2]["Info"]["VWG_NAAM"]).add_to(m)
    
    else:
        folium.PolyLine(line, color = "blue", weight = 2, popup = edge[2]["Info"]["VWG_NAAM"]).add_to(m)

m

### Create locks
We can see on the maps that there are locks on the graph, but that the information on the locks is limited. The following edges represent locks (NB: click the edges to show info:

- Voorhavens Jachtensluis
- Voorhavens Volkeraksluizen

These edges will be replaced by two lock elements. The Jachtensluizen are mainly designed for yachts and pleasure craft, and have a length of 135 meters and a width of 16 meters. The Volkeraksluizen have three chambers each with a length of 330 meters and a width of 24 meters. For now we'll assume that both locks have a maximum allowable draught of 4.5 meters.

Additional information on the locks can be found on the [Rijkswaterstand website (link in Dutch)](https://www.rijkswaterstaat.nl/water/waterbeheer/bescherming-tegen-het-water/waterkeringen/deltawerken/volkeraksluizen.aspx). 

In [46]:
lock_nr_1 = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Volkerak - 1", 
                        node_1 = "(4.430289, 51.700047)", node_2 = "(4.392555, 51.681251)",
                        lock_length = 330, lock_width = 24, lock_depth = 4.5, 
                        doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

lock_nr_2 = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Volkerak - 2", 
                        node_1 = "(4.430289, 51.700047)", node_2 = "(4.392555, 51.681251)",
                        lock_length = 330, lock_width = 24, lock_depth = 4.5, 
                        doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

lock_nr_3 = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Volkerak - 3", 
                        node_1 = "(4.430289, 51.700047)", node_2 = "(4.392555, 51.681251)",
                        lock_length = 330, lock_width = 24, lock_depth = 4.5, 
                        doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

# lock_test = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Jachtensluis", 
#                         node_1 = "(4.395179, 51.691512)", node_2 = "(4.408442, 51.700226)",
#                         lock_length = 330, lock_width = 24, lock_depth = 4.5, 
#                         doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)


In [47]:
for edge in FG_new.edges(data = True):
    if edge[2]["Info"]["VWG_NAAM"] == "Voorhavens Volkeraksluizen":
        # For testing, all locks have the water level at the right side
        lock_nr_1.water_level = "(4.430289, 51.700047)"
        lock_nr_2.water_level = "(4.430289, 51.700047)"
        lock_nr_3.water_level = "(4.430289, 51.700047)"
        
        # Add locks to the correct edge
        FG_new.edges[edge[0], edge[1]]["Lock"] = [lock_nr_1, lock_nr_2, lock_nr_3]
        
#Waarom pakken we hier het rechter water level
#En waar staat de informatie over water levels van de verschillende edges

### Load vessel database

In [48]:
# NB: This database needs to be triple checked (!!!)
# use the 'Vessel database v2.ipynb' notebook to generate 'Vessel-database-2.csv'
# for every value add a traceable source
vessel_db = pd.read_csv(location_vessel_database)
vessel_db.head()

,VesselID,B,L,H_e,H_f,T_e,T_f,emissionfactor,P_installed,L_w,...,capacity_loaded,capacity_unloaded,speed_loaded,speed_unloaded,resistance_loaded,resistance_unloaded,is_loaded,avv_class,cemt_class,type
0,29a24858-4a51-11e9-9792-b469212bff5b,5.06,39.76,2.0,5.25,1.2,2.5,0.73,36.0,1.0,...,260.0,0.0,3.083,4.470,16919.0,9188.0,0.0,M1,I,Spits
1,dcc05ff4-4972-11e9-a543-b469212bff5b,6.42,52.26,2.0,6.10,1.4,2.6,0.72,118.0,1.0,...,422.5,0.0,3.361,4.638,28770.0,17524.0,0.0,M2,II,Kempenaar
2,e6cea50a-4972-11e9-a1f1-b469212bff5b,7.15,72.21,2.0,6.40,1.5,2.6,0.72,120.0,1.0,...,520.0,0.0,3.861,4.638,43935.0,22746.0,0.0,M3,III,Hagenaar
3,f0eb9740-4972-11e9-96ab-b469212bff5b,8.02,67.02,2.0,6.60,1.5,2.7,0.72,160.0,1.0,...,682.5,0.0,4.000,4.889,50582.0,28777.0,0.0,M4,III,Dortmund-Eems (L <= 74 m)
4,f5685128-4972-11e9-9bc8-b469212bff5b,8.18,80.56,2.0,6.40,1.5,2.7,0.72,323.0,2.0,...,812.5,0.0,3.916,5.027,54096.0,31794.0,0.0,M5,III,Verl. Dortmund (L > 74 m)


In [49]:
# renaming some of the vessel_db fields to align them with the code (NB: this alignment should be improved)
# in the end vessel_db should hold all (and not more than) the fields required by Vessel (see further down) 

# rename fields
vessel_db = vessel_db.rename(columns={
                                      'capacity_loaded': 'capacity'}
                                     )

# remove parameters that are not used (later we may want to add these via a mix-in)
del vessel_db['VesselID']
del vessel_db['own_weight']
del vessel_db['capacity_unloaded']
del vessel_db['speed_loaded']
del vessel_db['speed_unloaded']
del vessel_db['is_loaded']
del vessel_db['avv_class']
del vessel_db['cemt_class']
del vessel_db['resistance_loaded']
del vessel_db['resistance_unloaded']
del vessel_db['emissionfactor']

In [50]:
# add fields that are necessary for the energy calculation (NB: now added blanket values, may need to be calculated)
vessel_db['C_b'] = 0.922218379546862
vessel_db['C_BB'] = 0.185

In [51]:
# a table with these fields is currently what the calculations need as input 
vessel_db.head()

,B,L,H_e,H_f,T_e,T_f,P_installed,L_w,capacity,type,C_b,C_BB
0,5.06,39.76,2.0,5.25,1.2,2.5,36.0,1.0,260.0,Spits,0.922218,0.185
1,6.42,52.26,2.0,6.10,1.4,2.6,118.0,1.0,422.5,Kempenaar,0.922218,0.185
2,7.15,72.21,2.0,6.40,1.5,2.6,120.0,1.0,520.0,Hagenaar,0.922218,0.185
3,8.02,67.02,2.0,6.60,1.5,2.7,160.0,1.0,682.5,Dortmund-Eems (L <= 74 m),0.922218,0.185
4,8.18,80.56,2.0,6.40,1.5,2.7,323.0,2.0,812.5,Verl. Dortmund (L > 74 m),0.922218,0.185


### Updated mixin classes

In [52]:
# Perhaps it would be good to include a V_e and a V_f later

class VesselProperties:
    """Mixin class: Something that has vessel properties

    type: can contain info on vessel type (avv class, cemt_class or other)
    B: vessel width
    L: vessel length
    H_e: vessel height unloaded
    H_f: vessel height loaded
    T_e: draught unloaded
    T_f: draught loaded

    Add information on possible restrictions to the vessels, i.e. height, width, etc.
    """

    def __init__(
        self,
        type,
        B,
        L,
        H_e,
        H_f,
        T_e,
        T_f,
        *args,
        **kwargs
        ):
        super().__init__(*args, **kwargs)

        """Initialization"""
        self.type = type

        self.B = B
        self.L = L

        self.H_e = H_e
        self.H_f = H_f                      

        self.T_e = T_e
        self.T_f = T_f

    @property
    def H(self):
        """ Calculate current height based on filling degree """

        return (
            self.filling_degree * (self.H_f - self.H_e)
            + self.H_e
        )

    @property
    def T(self):
        """ Calculate current draught based on filling degree
        
        Here we should implement the rules from Van Dorsser et al 
        """

        return (
            self.filling_degree * (self.T_f - self.T_e)
            + self.T_e
        )

    def get_route(
        self,
        origin,
        destination,
        graph=None,
        minWidth=None,
        minHeight=None,
        minDepth=None,
        randomSeed=4,
    ):
        """ Calculate a path based on vessel restrictions """

        graph = graph if graph else self.env.FG
        minWidth = minWidth if minWidth else 1.1 * self.B
        minHeight = minWidth if minHeight else 1.1 * self.H
        minDepth = minWidth if minDepth else 1.1 * self.T

        # Check if information on restrictions is added to the edges
        random.seed(randomSeed)
        edge = random.choice(list(graph.edges(data=True)))
        edge_attrs = list(edge[2].keys())

        # IMPROVE THIS TO CHECK ALL EDGES AND COMBINATIONS OF RESTRICTIONS

        if all(item in edge_attrs for item in ["Width", "Height", "Depth"]):
            edges = []
            nodes = []

            for edge in graph.edges(data=True):
                if (
                    edge[2]["Width"] >= minWidth
                    and edge[2]["Height"] >= minHeight
                    and edge[2]["Depth"] >= minDepth
                ):
                    edges.append(edge)

                    nodes.append(graph.nodes[edge[0]])
                    nodes.append(graph.nodes[edge[1]])

            subGraph = graph.__class__()

            for node in nodes:
                subGraph.add_node(
                    node["name"],
                    name=node["name"],
                    geometry=node["geometry"],
                    position=(node["geometry"].x, node["geometry"].y),
                )

            for edge in edges:
                subGraph.add_edge(edge[0], edge[1], attr_dict=edge[2])

            try:
                return nx.dijkstra_path(subGraph, origin, destination)
            except:
                raise ValueError(
                    "No path was found with the given boundary conditions."
                )

        # If not, return shortest path
        else:
            return nx.dijkstra_path(graph, origin, destination)


### Energy consumption calculations

In [53]:
# This mixin now contains the calculation rules as proposed by Loes Segers
# NB: all calculate values are added as a property to the vessel object (not sure this is needed/wise)
#     alternatively we could work more with inputs and returns to just get back the value you need

class ConsumesEnergy:
    """Mixin class: Something that consumes energy.

    P_installed: installed engine power [kW]
    C_b: block coefficient ('fullness') [-]
    C_BB: breadth coefficient (determines area of bulbous bow) [-]
    nu: kinematic viscosity [m^2/s]
    rho: density of the surrounding water [kg/m^3]
    g: gravitational accelleration [m/s^2]
    x: number of propellors [-]
    eta_0: open water efficiency of propellor [-]
    eta_r: relative rotative efficiency [-]
    eta_t: transmission efficiency [-]
    eta_g: gearing efficiency [-]
    c_stern: determines shape of the afterbody [-]
    one_k2: appendage resistance factor [-]
    """

    def __init__(
        self, 
        P_installed,
        C_b,   
        C_BB,
        nu = 1*10**(-6), # kinematic viscosity
        rho = 1000,
        g = 9.81,
        x = 2, # number of propellors
        eta_0 = 0.7,
        eta_r = 0.98,
        eta_t = 0.98,
        eta_g = 0.96,
        c_stern = 0,
        one_k2 = 1.5,
        *args, 
        **kwargs
        ):
        super().__init__(*args, **kwargs)

        """Initialization"""
        self.P_installed = P_installed
        self.C_b = C_b  
        self.C_BB = C_BB
        self.nu = nu 
        self.rho = rho
        self.g = g
        self.x = x
        self.eta_0 = eta_0
        self.eta_r = eta_r
        self.eta_t = eta_t
        self.eta_g = eta_g
        self.c_stern = c_stern
        self.one_k2 = one_k2
         
            
    def calculate_properties(self):
        """Calculate a number of basic properties"""
        self.C_M = 1.006 - 0.0056 * self.C_b ** (-3.56)          #Midship section coefficient
        self.C_wp = (1 + 2 * self.C_b) / 3                       #Waterplane coefficient
        self.C_p = self.C_b / self.C_M                           #Prismatic coefficient

        self.delta = self.C_b * self.L * self.B * self.T         #Water displacement

        self.lcb = -13.5 + 19.4 * self.C_p                                                     #longitudinal center of buoyancy
        self.L_R = self.L * (1 - self.C_p + (0.06 * self.C_p * self.lcb)/(4 * self.C_p - 1))   #parameter reflecting the length of the run

        self.A_BT = self.C_BB * self.B * self.T * self.C_M       #wet transverse sectional area of the bulbous bow
        self.A_T = 0.1 * self.B * self.T                         #transverse area of the transom
        
        #Total wet area
        self.S_T = self.L * (2 * self.T + self.B) * np.sqrt(self.C_M) * (0.453 + 0.4425 * self.C_b - 0.2862 * self.C_M - 0.003467 * (self.B / self.T) + 0.3696 * self.C_wp) + 2.38 * (self.A_BT / self.C_b)
       
        self.S_APP = 0.05 * self.S_T       #Wet area of appendages
        self.S_B = self.L * self.B         #Area of flat bottom

        self.D_s = 0.7 * self.T            #Diameter of the screw
        
        
    def calculate_frictional_resistance(self, V_0, h):
        """Section 2.2.1 Frictional resistance"""

        self.R_e = V_0 * self.L / self.nu        #Reynolds number
        self.D = h - self.T                      #distance from bottom ship to the bottom of the fairway
        
        #Friction coefficient in deep water
        self.Cf_0 = 0.075 / ((np.log10(self.R_e) - 2) ** 2)
        
        #Friction coefficient proposed, taking into account shallow water effects
        self.Cf_proposed = (0.08169 / ((np.log10(self.R_e) - 1.717) ** 2)) * (1 + (0.003998 / (np.log10(self.R_e) - 4.393)) * (self.D / self.L) ** (-1.083))

        #'a' is the coefficient needed to calculate the Katsui friction coefficient
        self.a = 0.042612 * np.log10(self.R_e) + 0.56725
        self.Cf_katsui = 0.0066577 / ((np.log10(self.R_e) - 4.3762) ** self.a)

        #The average velocity underneath the ship, taking into account the shallow water effect
        self.V_B = 0.4277 * V_0 * np.exp((h / self.T) ** (-0.07625))
        
        #cf_proposed cannot be applied directly, since a vessel also has non-horizontal wet surfaces that have to be taken
        #into account. Therefore, the following formula for the final friction coefficient 'C_f' is defined:
        self.C_f = self.Cf_0 + (self.Cf_proposed - self.Cf_katsui) * (self.S_B / self.S_T) * (self.V_B / V_0) ** 2

        #The total frictional resistance R_f [kN]:
        self.R_f = (self.C_f * 0.5 * self.rho * (V_0 ** 2) * self.S_T) / 1000 

        
    def calculate_viscous_resistance(self):
        """Section 2.2.2 Viscous resistance"""
        
        #c_14 accounts for the specific shape of the afterbody
        self.c_14 = 1 + 0.0011 * self.c_stern
        
        #the form factor (1+k1) describes the viscous resistance
        self.one_k1 = 0.93 + 0.487 * self.c_14 * ((self.B / self.L) ** 1.068) * ((self.T/self.L) ** 0.461) * ((self.L / self.L_R) ** 0.122) * (((self.L ** 3) / self.delta) ** 0.365) * ((1 - self.C_p) ** (-0.604))
        
    def calculate_appendage_resistance(self, V_0):
        """Section 2.2.3 Appendage resistance"""
        
        #Resistance resulting from wet area of appendages: R_APP [kN]
        self.R_APP = (0.5 * self.rho * (V_0 ** 2) * self.S_APP * self.one_k2 * self.C_f) / 1000
     
    
    def calculate_wave_resistance(self, V_0):
        """Section 2.2.4 Wave resistance"""

        self.F_n = V_0 / np.sqrt(self.g * self.L)          #Froude number

        #coefficient c_7
        if self.B / self.L < 0.11:
            self.c_7 = 0.229577 * (self.B / self.L) ** 0.33333
        elif self.B / self.L > 0.25:
            self.c_7 = 0.5 - 0.0625 * (self.L / self.B)
        else:
            self.c_7 = self.B / self.L
       
        #Half angle of entrance i_E
        self.i_E = 125.67 * (self.B / self.L) - 162.25 * (self.C_p ** 2) + 234.32 * (self.C_p ** 3) + 0.155087 * (self.lcb ** 3)
        self.c_1 = 2223105 * (self.c_7 ** 3.78613) * ((self.T / self.B) ** 1.07961) * ((90 - self.i_E) ** (-1.37565))

        #Parameter c_2 (depending on c_3) accounts for the reduction of wave resistance due to the action of the bulbous bow
        self.c_3 = 0.56 * (self.A_BT ** 1.5) / (self.B * self.T * (0.31 * np.sqrt(self.A_BT) + self.T - 0.5*self.T))
        self.c_2 = np.exp(-1.89 * np.sqrt(self.c_3))
        
        #c_5 accounts for the effect of the transom stern on the wave resistance.
        self.c_5 = 1 - 0.8 * self.A_T / (self.B * self.T * self.C_M)

        #coefficient c_16
        if self.C_p < 0.80:
            self.c_16 = 8.07981 * self.C_p - (13.8673 * self.C_p ** 2) + (6.984388 * self.C_p ** 3)
        else:
            self.c_16 = 1.73014 - 0.7067 * self.C_p

        self.m_1 = 0.0140407 * (self.L/self.T) - (1.75254 * self.delta**(1/3))/self.L - 4.79323 * (self.B/self.L) - self.c_16

        #coefficient c_15
        if (self.L**3)/self.delta < 512:
            self.c_15 = -1.69385
        elif (self.L**3)/self.delta > 1727:
            self.c_15 = 0
        else:
            self.c_15 = -1.69385 + (self.L / (self.delta**(1/3)) - 8.0) / 2.36

        self.m_2 = self.c_15 * (self.C_p**2) * np.exp(-0.1 * (self.F_n)**(-2))

        #coefficient lambda
        if self.L/self.B < 12:
            self.lmbda = 1.446 * self.C_p - 0.03 * (self.L/self.B)
        else:
            self.lmbda = 1.446 * self.C_p - 0.36

        #The wave resistance depends on all coefficients above: R_W [kN] 
        self.R_W = (self.c_1 * self.c_2 * self.c_5 * self.delta * self.rho * self.g * np.exp(self.m_1 * (self.F_n**-0.9) + self.m_2 * np.cos(self.lmbda * (self.F_n**(-2))))) / 1000 

    def calculate_residual_resistance(self, V_0):
        """Section 2.2.5 Residual resistance terms"""

        #Resistance resulting from the bulbouw bow: R_B [kN]
        self.P_B = 0.56 * np.sqrt(self.A_BT) / (self.T - 1.5 * 0.5*self.T)     #measure for emergence of bulbouw bow
        self.F_ni = V_0 / np.sqrt(self.g * (self.T - 0.5*self.T - 0.25*np.sqrt(self.A_BT)) + 0.15*(V_0**2))    #Froude number based on immersion

        self.R_B = (0.11 * np.exp(-3 * self.P_B**(-2)) * (self.F_ni**3) * (self.A_BT**1.5) * self.rho * self.g / (1 + (self.F_ni**2))) / 1000

        #Resistance due to immersed transom: R_TR [kN]
        self.F_nt = V_0 / np.sqrt(2 * self.g * self.A_T / (self.B + self.B * self.C_wp))    #Froude number based on transom immersion
        self.c_6 = 0.2 * (1 - 0.2 * self.F_nt)      #Assuming F_nt < 5, this is the expression for coefficient c_6

        self.R_TR = (0.5 * self.rho * (V_0**2) * self.A_T * self.c_6) / 1000 

        #Model-ship correlation resistance: R_A [kN]
        if self.T/self.L > 0.04:
            self.c_4 = 0.04
        else:
            self.c_4 = self.T / self.L

        self.C_A = 0.006 * ((self.L + 100) ** (-0.16)) - 0.00205 + 0.003 * np.sqrt(self.L / 7.5) * (self.C_b ** 4) * self.c_2 * (0.04 - self.c_4)

        self.R_A = (0.5 * self.rho * (V_0 ** 2) * self.S_T * self.C_A) / 1000 #kW

    def calculate_total_resistance(self, V_0, h):
        
        self.calculate_properties()
        self.calculate_frictional_resistance(V_0, h)
        self.calculate_viscous_resistance()
        self.calculate_appendage_resistance(V_0)
        self.calculate_wave_resistance(V_0)
        self.calculate_residual_resistance(V_0)
        
        #The total resistance R_tot [kN] = R_f * (1+k1) + R_APP + R_W + R_B + R_TR + R_A
        self.R_tot = self.R_f * self.one_k1 + self.R_APP + self.R_W + self.R_B + self.R_TR + self.R_A

    def calculate_total_power_required(self):
        """ Section 2.1 Total required power """

        #2.1.1 Required power for systems on board
        self.P_hotel = 0.081 * self.P_installed

        #2.1.2 Required power for propulsion

        #Effective Horse Power (EHP)
        self.P_EHP = self.V_B * self.R_tot
        
        #Calculation hull efficiency
        dw = np.zeros(101)              #velocity correction coefficient
        counter = 0 
        
        if self.F_n < 0.2:
            self.dw = 0
        else:
            self.dw = 0.1


        self.w = 0.11 * (0.16 / self.x) * self.C_b * np.sqrt((self.delta**(1/3)) / self.D_s) - self.dw   #wake fraction 'w'


        if self.x == 1:
            self.t = 0.6 * self.w * (1 + 0.67 * self.w)       #thrust deduction factor 't'
        else:
            self.t = 0.8 * self.w * (1 + 0.25 * self.w)

        self.eta_h = (1 - self.t) / (1 - self.w)    #hull efficiency eta_h

        #Delivered Horse Power (DHP)

        self.P_DHP = self.P_EHP / (self.eta_0 * self.eta_r * self.eta_h)

        #Brake Horse Power (BHP)
        self.P_BHP = self.P_DHP / (self.eta_t * self.eta_g)

        self.P_tot = self.P_hotel + self.P_BHP
        
        #Partial engine load (P_partial): needed in the 'Emission calculations' 
        self.P_partial = self.P_tot / self.P_installed

### Emission calculations

In [54]:
class EmissionCalculations:
    """Mixin class: Emissions that result from a certain energy consumption

    P_installed: installed engine power [kW]
    L_w = weight class (L1, L2 or L3) which determines the weibull function of the age of the engine
    """
    
    def __init__(
        self, 
        #P_installed,
        L_w,
        *args, 
        **kwargs
        ):
        super().__init__(*args, **kwargs)

        """Initialization"""
        #self.P_installed = P_installed
        self.L_w = L_w
        
        
    def calculate_engine_age(self):
        """Calculating the construction year of the engine, dependend on a Weibull function with 
        shape factor 'k', and scale factor 'lmb', which are determined by the weight class L_w"""
        
        #Determining which shape and scale factor to use, based on the weight class L_w = L1, L2 or L3
        if self.L_w == 1:     #Weight class L1
            self.k = 1.3
            self.lmb = 20.5
        if self.L_w == 2:     #Weight class L2
            self.k = 1.12
            self.lmb = 18.5
        if self.L_w == 3:     #Weight class L3
            self.k = 1.26
            self.lmb = 18.6
            
        #The age of the engine
        self.age = int(np.random.weibull(self.k)*self.lmb)
        
        #Current year
        self.year = 2020
        
        #Construction year of the engine
        self.c_year = self.year - self.age
        
        
    def emission_factors_general(self):
        """Calculating the general emission factors, based on construction year of the engine"""
        
        self.calculate_engine_age() #You need the values of c_year
        
        #The general emission factors of CO2, PM10 and NOX are based on the construction year of the engine
        
        if self.c_year < 1974:
            self.EM_CO2 = 756
            self.EM_PM10 = 0.6
            self.EM_NOX = 10.8
        if 1975 <= self.c_year <= 1979:
            self.EM_CO2 = 730
            self.EM_PM10 = 0.6
            self.EM_NOX = 10.6
        if 1980 <= self.c_year <= 1984:
            self.EM_CO2 = 714
            self.EM_PM10 = 0.6
            self.EM_NOX = 10.4
        if 1985 <= self.c_year <= 1989:
            self.EM_CO2 = 698
            self.EM_PM10 = 0.5
            self.EM_NOX = 10.1
        if 1990 <= self.c_year <= 1994:
            self.EM_CO2 = 698
            self.EM_PM10 = 0.4
            self.EM_NOX = 10.1
        if 1995 <= self.c_year <= 2002:
            self.EM_CO2 = 650
            self.EM_PM10 = 0.3
            self.EM_NOX = 9.4
        if 2003 <= self.c_year <= 2007:
            self.EM_CO2 = 635
            self.EM_PM10 = 0.3
            self.EM_NOX = 9.2
        if 2008 <= self.c_year <= 2019:
            self.EM_CO2 = 635
            self.EM_PM10 = 0.2
            self.EM_NOX = 7
        if self.c_year > 2019:
            if self.L_w == 1:
                self.EM_CO2 = 650
                self.EM_PM10 = 0.1
                self.EM_NOX = 2.9
            else:
                self.EM_CO2 = 603
                self.EM_PM10 = 0.0015
                self.EM_NOX = 2.4
        
        
    def correction_factors(self):
        """Calculating the correction factors that have to be applied to the general emission factors, 
        based on the partial load of the engine"""
        
        self.calculate_engine_age()  #You need the values of c_year
        self.calculate_total_power_required() #You need the P_partial values
        
        #Import the correction factors table
        self.corf = pd.read_excel (r'C:\Users\Loes\Documents\OpenTNSim\notebooks\correctionfactors.xlsx')
        
        for i in range(20):
        #If the partial engine load is smaller or equal to 5%, the correction factors corresponding to P_partial = 5% are assigned.
            if self.P_partial <= self.corf.iloc[0, 0]:
                self.corf_CO2 = self.corf.iloc[0, 5]
                self.corf_PM10 = self.corf.iloc[0, 6]
        
            #The NOX correction factors are dependend on the construction year of the engine and the weight class
                if self.c_year < 2008:
                    self.corf_NOX = self.corf.iloc[0, 1]   #<= CCR-1 class
                if 2008 <= self.c_year <= 2019:
                    self.corf_NOX = self.corf.iloc[0, 2]   #CCR-2 / Stage IIIa
                if self.c_year > 2019:
                    if self.L_w == 1:            #
                        self.corf_NOX = self.corf.iloc[0, 3]    #Stage V: IWP/IWA-v/c-3 class (vessels with P <300 kW: assumed to be weight class L1)
                    else:
                        self.corf_NOX = self.corf.iloc[0, 4]    #Stage V:IWP/IWA-v/c-4 class (vessels with P >300 kw: assumed to be weight class L2-L3)
        
        #If the partial engine load is greater than 5%:
            #It is determined inbetween which two percentages the partial load lies
            #The correction factor is determined by means of linear interpolation
            
        if self.corf.iloc[i, 0] < self.P_partial <= self.corf.iloc[i + 1, 0]:
            self.corf_CO2 = ((self.P_partial - self.corf.iloc[i, 0]) * (self.corf.iloc[i+1, 5] - self.corf.iloc[i, 5])) / (self.corf.iloc[i+1, 0] - self.corf.iloc[i, 0]) + self.corf.iloc[i, 5]
            self.corf_PM10 = ((self.P_partial - self.corf.iloc[i, 0]) * (self.corf.iloc[i+1, 6] - self.corf.iloc[i, 6])) / (self.corf.iloc[i+1, 0] - self.corf.iloc[i, 0]) + self.corf.iloc[i, 6]
            
            if self.c_year < 2008:
                self.corf_NOX = ((self.P_partial - self.corf.iloc[i, 0]) * (self.corf.iloc[i+1, 1] - self.corf.iloc[i, 1])) / (self.corf.iloc[i+1, 0] - self.corf.iloc[i, 0]) + self.corf.iloc[i, 1]
            if 2008 <= self.c_year <= 2019:
                self.corf_NOX = ((self.P_partial - self.corf.iloc[i, 0]) * (self.corf.iloc[i+1, 2] - self.corf.iloc[i, 2])) / (self.corf.iloc[i+1, 0] - self.corf.iloc[i, 0]) + self.corf.iloc[i, 2]
            if self.c_year > 2019:
                if self.L_w ==1:
                    self.corf_NOX = ((self.P_partial - self.corf.iloc[i, 0]) * (self.corf.iloc[i+1, 3] - self.corf.iloc[i, 3])) / (self.corf.iloc[i+1, 0] - self.corf.iloc[i, 0]) + self.corf.iloc[i, 3]
                else:
                    self.corf_NOX = ((self.P_partial - self.corf.iloc[i, 0]) * (self.corf.iloc[i+1, 4] - self.corf.iloc[i, 4])) / (self.corf.iloc[i+1, 0] - self.corf.iloc[i, 0]) + self.corf.iloc[i, 4]
                
    def calculate_emission_factors_total(self):
        """Calculating the total emission factors, by multiplying the general emission factors with the 
        correction factors"""
            
        self.emission_factors_general()  #You need the values of the general emission factors of CO2, PM10, NOX
        self.correction_factors() #You need the correction factors of CO2, PM10, NOX
        
        
        #The total emission factor is calculated by multiplying the general emission factor (EM_CO2 / EM_PM10 / EM_NOX)
        # By the correction factor (corf_CO2 / corf_PM10 / corf_NOX)
        
        self.Emf_CO2 = self.EM_CO2 * self.corf_CO2
        self.Emf_PM10 = self.EM_PM10 * self.corf_PM10
        self.Emf_NOX = self.EM_NOX * self.corf_NOX

In [55]:
#corf = pd.read_excel (r'C:\Users\Loes\Documents\OpenTNSim\notebooks\correctionfactors.xlsx')
#corf.head()

### Create a vessel object with the new mixins, and use this in the vessel generator

In [56]:
# Vessel type
Vessel = type('Vessel', 
              (core.Identifiable,      # so we can identify an individual vessel (by name and id)
               VesselProperties,       # so that we can provide vessel properties
               core.Movable,           # so we van make the vessel move
               core.HasContainer,      # so that we can indicate the amount of cargo it carries
               core.HasResource,       # so that a vessel be requested to do something
               core.Routeable,         # so that you can provide a route that the vessel will follow
               ConsumesEnergy,         # so that you can calculate energy used on route
               EmissionCalculations),  # so that you can calculate the emissions on the route
              {})

In [57]:
generator = model.VesselGenerator(Vessel, vessel_db)

### Run simulation

In [58]:
# NB simpy environment was already initiated 
env.FG = FG_new

In [59]:
# Randomly draw a number of vessels (with random origins and destinations)
vessels = []

# Add 10 vessels to the simulation
for i in range(10):
    random_1 = random.choice(list(env.FG))
    random_2 = random.choice(list(env.FG))
    path = nx.dijkstra_path(env.FG, random_1, random_2)
    
    vessel = generator.generate(env, "Vessel " + str(i))
    vessel.route = path
    vessel.geometry = nx.get_node_attributes(env.FG, "geometry")[vessel.route[0]]
    
    vessels.append(vessel)
    
    # Add the movements of the vessel to the simulation
    env.process(vessel.move())
#     env.process(start(env, vessel))

In [60]:
# Run simulation
env.run()

### Check results

In [61]:
# pick a vessel from the vessels list and inspect the results (for now it does not seem the locks are working)
for vessel in vessels:
    print(vessel.name)
    df = pd.DataFrame.from_dict(vessel.log)

    # print all messages
    for index, row in df.iterrows():
        print(row['Message'])

Vessel 0
Sailing from node (4.415027, 51.704766) to node (4.423482, 51.710607) start
Sailing from node (4.415027, 51.704766) to node (4.423482, 51.710607) stop
Sailing from node (4.423482, 51.710607) to node (4.436174, 51.702279) start
Sailing from node (4.423482, 51.710607) to node (4.436174, 51.702279) stop
Sailing from node (4.436174, 51.702279) to node (4.430289, 51.700047) start
Sailing from node (4.436174, 51.702279) to node (4.430289, 51.700047) stop
Passing lock start
Passing lock stop
Vessel 1
Sailing from node (4.383376, 51.716016) to node (4.401283, 51.713412) start
Sailing from node (4.383376, 51.716016) to node (4.401283, 51.713412) stop
Sailing from node (4.401283, 51.713412) to node (4.423482, 51.710607) start
Sailing from node (4.401283, 51.713412) to node (4.423482, 51.710607) stop
Sailing from node (4.423482, 51.710607) to node (4.415027, 51.704766) start
Sailing from node (4.423482, 51.710607) to node (4.415027, 51.704766) stop
Sailing from node (4.415027, 51.704766)

### Calculate energy use

In [62]:
# Define a EnergyCalculation class (this class postprocesses all results and calculates the energy (kWh))
class EnergyCalculation:
    """
    Add information on energy use and effects on energy use.
    """

    def __init__(self, vessel, *args, **kwargs):
        super().__init__(*args, **kwargs)

        """Initialization"""
        self.vessel = vessel
        
        self.energy_use = {"time_start": [], 
                           "time_stop": [], 
                           "edge_start": [],
                           "edge_stop": [],
                           "total_energy": [], 
                           "total_emission_CO2": [],
                           "total_emission_PM10": [], 
                           "total_emission_NOX": [], 
                           "stationary": []}
        
        self.co2_footprint = {"total_footprint": 0, "stationary": 0}
        self.mki_footprint = {"total_footprint": 0, "stationary": 0}

    def calculate_energy_consumption(self):
        """Calculation of energy consumption based on total time in system and properties"""

        def calculate_distance(geom_start, geom_stop):
            """method to calculate the distance in meters between two geometries"""
            wgs84 = pyproj.Geod(ellps='WGS84')

            # distance between two points
            return int(wgs84.inv(geom_start.x, geom_start.y,
                                 geom_stop.x,  geom_stop.y) [2])
        
        # log messages that are related to locking
        stationary_phase_indicator = [
            "Waiting to enter waiting area stop",
            "Waiting in waiting area stop",
            "Waiting in line-up area stop",
            "Passing lock stop",
        ]
        
        # extract relevant elements from the vessel log
        times = self.vessel.log["Timestamp"]
        messages = self.vessel.log["Message"]
        geometries = self.vessel.log["Geometry"]

        # now walk past each logged event (each 'time interval' in the log corresponds to an event)
        for i in range(len(times) - 1):
            # determine the time associated with the logged event (how long did it last)
            delta_t = (times[i + 1] - times[i]).seconds
            
            if delta_t != 0:
                # append time information to the variables for the dataframe
                self.energy_use["time_start"].append(times[i])
                self.energy_use["time_stop"].append(times[i + 1])
                
                # append geometry information to the variables for the dataframe
                self.energy_use["edge_start"].append(geometries[i])
                self.energy_use["edge_stop"].append(geometries[i + 1])
                
                #print(i)
                #print(FG_new.edges[('({:f}, {:f})'.format(geometries[i].x, geometries[i].y),'({:f}, {:f})'.format(geometries[i+1].x, geometries[i+1].y))])

                # calculate the distance travelled and the associated velocity 
                distance = calculate_distance(geometries[i], geometries[i + 1])
                V_0 = distance / delta_t
                
                # printstatements to check the output (can be removed later)
                print('delta_t: {} s'. format(delta_t))
                print('distance: {} m'. format(distance))
                print('velocity: {} m/s'. format(V_0))
                
                # we use the calculated velocity to determine the resistance (at h=10 for now) and power required
                self.vessel.calculate_total_resistance(V_0, h=10)
                self.vessel.calculate_total_power_required()
                self.vessel.calculate_emission_factors_total()

                if messages[i + 1] in stationary_phase_indicator:  # if we are in a stationary stage only log P_hotel
                    #Energy consumed per time step delta_t in the stationary stage
                    energy_delta = self.vessel.P_hotel * delta_t / 3600  # kJ/3600 = kWh
                    
                    #Emissions CO2, PM10 and NOX - emitted in the stationary stage per time step delta_t, consuming 'energy_delta' kWh
                    emission_delta_CO2 = self.vessel.Emf_CO2 * energy_delta # in g
                    emission_delta_PM10 = self.vessel.Emf_PM10 * energy_delta # in g
                    emission_delta_NOX = self.vessel.Emf_NOX * energy_delta # in g
                    
                    self.energy_use["total_energy"].append(energy_delta)
                    self.energy_use["stationary"].append(energy_delta)
                    self.energy_use["total_emission_CO2"].append(emission_delta_CO2)
                    self.energy_use["total_emission_PM10"].append(emission_delta_PM10)
                    self.energy_use["total_emission_NOX"].append(emission_delta_NOX)

                else:  # otherwise log P_tot
                    #Energy consumed per time step delta_t in the propulsion stage
                    energy_delta = self.vessel.P_tot * delta_t / 3600  # kJ/3600 = kWh
                   
                    #Emissions CO2, PM10 and NOX - emitted in the propulsion stage per time step delta_t, consuming 'energy_delta' kWh
                    emission_delta_CO2 = self.vessel.Emf_CO2 * energy_delta #Energy consumed per time step delta_t in the stationary phase a # in g
                    emission_delta_PM10 = self.vessel.Emf_PM10 * energy_delta # in g
                    emission_delta_NOX = self.vessel.Emf_NOX * energy_delta # in g
    
                    self.energy_use["total_energy"].append(energy_delta)
                    self.energy_use["stationary"].append(0)
                    self.energy_use["total_emission_CO2"].append(emission_delta_CO2)
                    self.energy_use["total_emission_PM10"].append(emission_delta_PM10)
                    self.energy_use["total_emission_NOX"].append(emission_delta_NOX)
        
        # TODO: er moet hier een heel aantal dingen beter worden ingevuld
        # - de kruissnelheid is nu nog per default 1 m/s (zie de Movable mixin). Eigenlijk moet in de 
        #   vessel database ook nog een speed_loaded en een speed_unloaded worden toegevoegd. 
        # - er zou nog eens goed gekeken moeten worden wat er gedaan kan worden rond kustwerken 
        # - en er is nog iets mis met de snelheid rond een sluis

    def plot(self):
        
        import folium

        df = pd.DataFrame.from_dict(energycalculation.energy_use)

        m = folium.Map(location=[51.7, 4.4], zoom_start = 12)

        line = []
        for index, row in df.iterrows():
            line.append((row["edge_start"].y, row["edge_start"].x))

        folium.PolyLine(line, weight = 4).add_to(m)

        return m

In [63]:
energycalculation = EnergyCalculation(vessels[3])

In [64]:
energycalculation.calculate_energy_consumption()

delta_t: 3066 s
distance: 3066 m
velocity: 1.0 m/s


AttributeError: 'Vessel' object has no attribute 'corf_CO2'

In [ ]:
energycalculation.plot()

In [ ]:
pd.DataFrame.from_dict(energycalculation.energy_use)

In [ ]:
import folium

df = pd.DataFrame.from_dict(energycalculation.energy_use)

m = folium.Map(location=[51.7, 4.4], zoom_start = 12)

line = []
for index, row in df.iterrows():
    line.append((row["edge_start"].y, row["edge_start"].x))

folium.PolyLine(line, weight = 2).add_to(m)

m

### Show loginfo of the locks

In [ ]:
pd.DataFrame.from_dict(lock_nr_1.log)

In [ ]:
pd.DataFrame.from_dict(lock_nr_2.log)

In [ ]:
pd.DataFrame.from_dict(lock_nr_3.log)

### Visualise on Google Earth

In [ ]:
plot.vessel_kml(env, [vessels[3]])

### Sandbox

In [ ]:
vessels[3].__dict__